# Quantum transport mini-projects (ASE + tight-binding + NEGF)

These notebooks are designed for **hands-on, live coding**: build a small graphene-based device, assemble a Hamiltonian, and compute bandstructure/DOS/transmission **without** QuantumATK/SIESTA.

**Core packages**: `numpy`, `scipy`, `matplotlib`, `ase`  
**Optional**: `sisl` (for structure I/O, Hamiltonian export, and extra analysis)

> Tip: Run on a laptop. Keep systems small (≤ a few hundred atoms) for fast feedback.

In [ ]:
# Colab / local: install essentials
# If you're local and already have these, you can comment this out.
!pip -q install ase scipy matplotlib

# Optional (uncomment if you want to use SISL utilities)
# !pip -q install sisl

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from dataclasses import dataclass
from typing import Tuple, Optional, List

from ase.build import graphene_nanoribbon
from ase.visualize import view

In [ ]:
import scipy.linalg as la

def fermi(E, mu=0.0, kBT=0.02585):
    """Fermi-Dirac distribution. kBT in eV."""
    x = (E - mu) / max(kBT, 1e-12)
    # numerically stable
    return np.where(x>50, 0.0, np.where(x<-50, 1.0, 1.0/(1.0+np.exp(x))))

def dfermi_dE(E, mu=0.0, kBT=0.02585):
    """Derivative -df/dE (positive)."""
    f = fermi(E, mu, kBT)
    return (f*(1-f))/max(kBT, 1e-12)

def sancho_rubio_surface_gf(E, H00, H01, eta=1e-6, maxiter=200, tol=1e-12):
    """Surface Green's function for a semi-infinite lead using Sancho-Rubio decimation.

    H00: onsite block (n x n)
    H01: coupling to next principal layer (n x n)
    """
    z = (E + 1j*eta)
    n = H00.shape[0]
    I = np.eye(n, dtype=complex)
    a = H01.copy().astype(complex)
    b = H01.conj().T.copy().astype(complex)
    e = H00.copy().astype(complex)
    g = la.inv(z*I - e)
    for _ in range(maxiter):
        ag = a @ g
        bg = b @ g
        e_new = e + ag @ b + bg @ a
        a_new = ag @ a
        b_new = bg @ b
        g_new = la.inv(z*I - e_new)
        if np.max(np.abs(g_new - g)) < tol:
            g = g_new
            break
        e, a, b, g = e_new, a_new, b_new, g_new
    return g

def self_energy(E, H00, H01, eta=1e-6):
    gs = sancho_rubio_surface_gf(E, H00, H01, eta=eta)
    return H01.conj().T @ gs @ H01

def transmission(E, Hc, SigmaL, SigmaR, eta=1e-6):
    n = Hc.shape[0]
    I = np.eye(n, dtype=complex)
    G = la.inv((E+1j*eta)*I - Hc - SigmaL - SigmaR)
    GL = 1j*(SigmaL - SigmaL.conj().T)
    GR = 1j*(SigmaR - SigmaR.conj().T)
    T = np.real(np.trace(GL @ G @ GR @ G.conj().T))
    return T, G, GL, GR

In [ ]:
def build_tb_from_positions(pos, cutoff=1.7, t=-2.7, onsite=0.0):
    """Build a simple nearest-neighbor TB Hamiltonian from atomic positions.
    Uses a distance cutoff to define neighbors.
    """
    pos = np.asarray(pos)
    n = len(pos)
    H = np.zeros((n, n), dtype=float)
    np.fill_diagonal(H, onsite)
    # brute force neighbor search (fine for small n)
    for i in range(n):
        for j in range(i+1, n):
            d = np.linalg.norm(pos[i]-pos[j])
            if d < cutoff:
                H[i, j] = t
                H[j, i] = t
    return H

def block_tridiagonal_device(H0, H1, ncells):
    """Create central Hamiltonian for ncells repeats of principal layer.
    H0: intra-layer (n x n), H1: inter-layer coupling (n x n)
    Returns Hc (n*ncells x n*ncells)
    """
    n = H0.shape[0]
    Hc = np.zeros((n*ncells, n*ncells), dtype=float)
    for c in range(ncells):
        sl = slice(c*n,(c+1)*n)
        Hc[sl, sl] = H0
        if c < ncells-1:
            sl2 = slice((c+1)*n,(c+2)*n)
            Hc[sl, sl2] = H1
            Hc[sl2, sl] = H1.T
    return Hc

def add_adatom_onsite(H, idx, delta=1.0):
    """Mimic an adatom by shifting onsite energy at a site."""
    H = H.copy()
    H[idx, idx] += delta
    return H

def add_anderson_disorder(H, W=0.5, rng=None):
    """Random onsite disorder in [-W/2, W/2]."""
    rng = np.random.default_rng() if rng is None else rng
    H = H.copy()
    H[np.diag_indices_from(H)] += rng.uniform(-W/2, W/2, size=H.shape[0])
    return H

# Project 2 — Temperature-dependent conductance from transmission

Goal: connect **T(E)** to measurable **G(T)** using Landauer.

In linear response:

\[ G(T) = \frac{2e^2}{h} \int dE\; T(E)\left(-\frac{\partial f}{\partial E}\right) \]

We will compute **thermal broadening** and explore how it smooths conductance steps.

In [ ]:
!pip -q install ase scipy matplotlib

In [ ]:
import numpy as np, matplotlib.pyplot as plt
from ase.build import graphene_nanoribbon

## Build a ribbon + compute transmission

In [ ]:
atoms = graphene_nanoribbon(width=9, length=1, type='armchair', saturated=False, vacuum=8.0, periodic=True)
H00, H01 = principal_layer_blocks(atoms, cutoff=1.7, t=-2.7, onsite=0.0)

Es = np.linspace(-1.5, 1.5, 801)
T_E = device_transmission(H00, H01, ncells=8, Es=Es, eta=2e-5)

plt.figure()
plt.plot(Es, T_E)
plt.xlabel("Energy (eV)")
plt.ylabel("T(E)")
plt.title("Ballistic transmission")
plt.show()

## Compute G(T) (up to the quantum 2e^2/h)

We'll report **dimensionless conductance** g(T)=G/(2e^2/h).

In [ ]:
def g_of_T(Es, T_E, kBT):
    # numerical integral
    w = dfermi_dE(Es, mu=0.0, kBT=kBT)
    return np.trapz(T_E * w, Es)

kBTs = np.array([0.001, 0.005, 0.01, 0.02, 0.05, 0.1])  # eV
gTs = np.array([g_of_T(Es, T_E, kBT) for kBT in kBTs])

plt.figure()
plt.plot(kBTs, gTs, marker="o")
plt.xlabel("kBT (eV)")
plt.ylabel(r"$g(T)=G/(2e^2/h)$")
plt.title("Temperature-dependent conductance (thermal smearing)")
plt.show()

## Conductance vs gate voltage (chemical potential)

Shift the Fermi level (like a gate) and see how temperature smears the steps.

In [ ]:
def g_vs_mu(Es, T_E, mu, kBT):
    w = dfermi_dE(Es, mu=mu, kBT=kBT)
    return np.trapz(T_E * w, Es)

mus = np.linspace(-0.6, 0.6, 241)
for kBT in [0.005, 0.02, 0.05]:
    gmu = [g_vs_mu(Es, T_E, mu, kBT) for mu in mus]
    plt.plot(mus, gmu, label=f"kBT={kBT:.3f} eV")
plt.xlabel("Chemical potential μ (eV)")
plt.ylabel(r"$g(\mu, T)$")
plt.title("Conductance vs gate at different temperatures")
plt.legend()
plt.show()

## Extension: crude “phonon-like” broadening

Real temperature effects can also include **inelastic scattering** (phonons). A simple toy model is to add an
energy broadening parameter `eta` (larger `eta` ≈ stronger dephasing). Try varying `eta` in `device_transmission`.

**Exercise**: Plot g(T) for multiple `eta` values and discuss which effects look like *thermal smearing* vs *dephasing*.